In [1]:
from p2pfl.node import Node
import time

In [2]:
n=2
nodes = []
for i in range(n):
    node = Node(host="localhost")
    node.start()
    nodes.append(node)
    


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmpwdc84jxa
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmpwdc84jxa/_remote_module_non_sriptable.py
INFO:root:Nodo a la escucha en localhost 56271
INFO:root:Nodo a la escucha en localhost 56272


In [3]:
for i in range(len(nodes)-1):
    nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
    time.sleep(0.1)


INFO:root:Conexión aceptada con ('localhost', 56272)


In [4]:
# Check if they are connected
for node in nodes:
    print(len(node.neightboors) == n-1)
    

True
True


In [5]:
nodes[0].set_start_learning(5)

INFO:root:Broadcasting start learning...
INFO:root:Training...
DEBUG:root:Processing message: ['START_LEARNING', '5']
INFO:root:Broadcasting model to all clients...
INFO:root:Training...
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (1/1)
INFO:root:Model added (1/1)
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Round 1 of 5 finished.
INFO:root:Training...
INFO:root:Round 1 of 5 finished.
INFO:root:Training...
INFO:root:Broadcasting model to all clients...
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (1/1)
INFO:root:Agregating models.
INFO:root:Model added (1/1)
INFO:root:Agregating models.
INFO:root:Round 2 of 5 finished.
INFO:root:Training...
INFO:root:Broadcasting model to all clients...
INFO:root:Round 2 of 5 finished.
INFO:root:Training...
INFO:root:Model added (1/1)
INFO:root:Broadcasting model to all clients...
INFO:root:Agregating models.
INFO:root:Model added (1/1)
INFO:root:Round 3 of 5 finished.
INFO:root:Ag

In [6]:
for node in nodes:
    print(node.learner.get_parameters())

OrderedDict([('l1.weight', tensor([[-0.0196, -0.0010, -0.0119,  ..., -0.0033, -0.0102, -0.0117],
        [-0.0024,  0.0259, -0.0151,  ..., -0.0134, -0.0134, -0.0005],
        [-0.0238,  0.0254, -0.0261,  ..., -0.0043, -0.0004, -0.0026],
        ...,
        [-0.0262, -0.0050, -0.0116,  ...,  0.0127,  0.0066,  0.0004],
        [-0.0014, -0.0011, -0.0162,  ...,  0.0201, -0.0081,  0.0114],
        [-0.0005,  0.0065, -0.0122,  ..., -0.0090,  0.0066, -0.0005]])), ('l1.bias', tensor([-0.0260, -0.0038,  0.0327, -0.0058,  0.0317, -0.0206,  0.0020, -0.0032,
         0.0059, -0.0098, -0.0151,  0.0111,  0.0302, -0.0240, -0.0087, -0.0106,
         0.0178, -0.0080,  0.0298,  0.0117, -0.0056, -0.0010,  0.0036, -0.0018,
         0.0075,  0.0142, -0.0107,  0.0064, -0.0032, -0.0096,  0.0026, -0.0090,
         0.0241,  0.0176,  0.0187, -0.0080, -0.0036, -0.0013,  0.0142,  0.0247,
         0.0264,  0.0313, -0.0120,  0.0089, -0.0125, -0.0037, -0.0011, -0.0069,
         0.0081, -0.0170, -0.0019,  0.0005,  

DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']


In [7]:
for node in nodes:
    print(node.round)

1
1
1
1
1


In [11]:
nodes[0].set_stop_learning()

INFO:root:Stopping learning
INFO:root:Stopping learning
INFO:root:Stopping learning
INFO:root:Stopping learning
INFO:root:Stopping learning


In [ ]:
from p2pfl.node import Node
from p2pfl.learning.model import MyNodeLearning
from p2pfl.agregator import FedAvg    
from collections import OrderedDict
import torch
import time

def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(host="localhost")
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f
            
        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.2) #Esperar por la asincronía
        
        
test_convergence((2,3))

INFO:root:Nodo a la escucha en localhost 58355
INFO:root:Nodo a la escucha en localhost 58356
INFO:root:Conexión aceptada con ('localhost', 58356)
INFO:root:Broadcasting start learning...
INFO:root:Training...
INFO:root:Training...
INFO:root:Broadcasting model to all clients...
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (1/1)
INFO:root:Agregating models.
INFO:root:Round 1 of 3 finished.
INFO:root:Training...
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (1/1)
INFO:root:Agregating models.
INFO:root:Round 1 of 3 finished.
INFO:root:Training...
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (1/1)
INFO:root:Agregating models.
INFO:root:Round 2 of 3 finished.
INFO:root:Training...
INFO:root:Broadcasting model to all clients...
INFO:root:Error executing callback: invalid load key, '\xe8'.
ERROR:root:invalid load key, '\xe8'.
Traceback (most recent call last):
  File "/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



DEBUG:root:Closed connection with ('localhost', 58356)
DEBUG:root:Too mucho errors. ('localhost', 58355)
DEBUG:root:Last error: b''
DEBUG:root:Buffer: b''
DEBUG:root:Closed connection with ('localhost', 58355)


In [6]:
a = b'000000001'
pos = a.find(b'1')

In [4]:
list = []
list.append(False)

list.all()

AttributeError: 'list' object has no attribute 'all'

In [ ]:
00000000
000000001000